In [ ]:
%pwd

: 

In [2]:
import os
os.chdir("../")

In [3]:
%pwd

'C:\\Users\\venkatarambabuvaddi\\Documents\\Projects\\Medical-chat-bot'

In [4]:
from langchain_community.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

ModuleNotFoundError: No module named 'langchain_community'

In [5]:
def load_pdf_file(data):
    loader=DirectoryLoader(data,
                           glob="*.pdf",
                           loader_cls=PyPDFLoader)
    document = loader.load()
    return document

In [ ]:
extract_data = load_pdf_file(data=r'C:\Users\venkatarambabuvaddi\Documents\Projects\Medical-chat-bot\Data')

In [ ]:
extract_data

In [ ]:
#Split te data into Text chunks.
def text_split(extract_data):
    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=10)
    text_chunks = splitter.split_documents(extract_data)
    return text_chunks


In [ ]:
text_chunks = text_split(extract_data)
print("Length of text Chunks",len(text_chunks))

In [ ]:
text_chunks

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

In [ ]:
#download the Embedding from Hugging face
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [ ]:
embeddings = download_hugging_face_embeddings()

In [ ]:
query_result = embeddings.embed_query("Hello world")
print("Length",len(query_result))

In [ ]:
query_result

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv(r'C:\Users\venkatarambabuvaddi\Documents\Projects\Medical-chat-bot\.env')

In [ ]:
import os

API_KEY=os.getenv("API_KEY")
OPENAI_KEY=os.getenv("OPENAI_API_KEY")

print("API_KEY:",API_KEY)
print("OPENAI_KEY:",OPENAI_KEY)

In [ ]:
from groq import Groq

client = Groq(api_key=API_KEY)
completion = client.chat.completions.create(
    model="openai/gpt-oss-20b",
    messages=[
      {
        "role": "user",
        "content": "Whats is acne"
      }
    ],
    temperature=1,
    max_completion_tokens=8192,
    top_p=1,
    reasoning_effort="medium",
    stream=False,
    stop=None
)

print(completion.choices[0].message)



In [ ]:
# 4. Store chunks in FAISS vector DB
from langchain_community.vectorstores import FAISS

docsearch = FAISS.from_documents(
    documents=text_chunks,
    embedding=embeddings
)

print("FAISS index created successfully!")

In [ ]:
index_path = "faiss_index"

if os.path.exists(index_path):
    # Load existing FAISS index
    docsearch = FAISS.load_local(
        index_path,
        embeddings,
        allow_dangerous_deserialization=True)
    print("✅ Loaded existing FAISS index.")

In [ ]:
'''from langchain_community.vectorstores import FAISS

docsearch.save_local("faiss_index")   # folder name will be created

 Later: Load index back
docsearch = FAISS.load_local(
    "faiss_index",
    embeddings,
    allow_dangerous_deserialization=True
)

print("FAISS index loaded successfully!")'''


In [ ]:
docsearch

In [ ]:

# Convert FAISS into a retriever
retriever = docsearch.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)

print("Retriever ready!")

In [ ]:
retrived_docs = retriever.invoke("What is acne")
print("Retriever retrieved successfully!")

In [ ]:
retrived_docs

In [ ]:
from langchain_openai import ChatOpenAI
import os

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.7,
    api_key=API_KEY  # pass directly
)
print("OpenAI API key is ",OPENAI_API_KEY)

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.prompts import ChatPromptTemplate


system_prompt = (
"You are an assistant for question-answering tasks. "
"Use the following pieces of retrieved context to answer"
"the question. If you don't know the answer, say that you"
"don't know. Use three sentences maximum and keep the"
"answer concise."
"\n\n"
"{context}"
)
prompt = ChatPromptTemplate.from_messages(
[
("system", system_prompt),
("human", "{input}"),
    ]
)

In [ ]:
question_answer_chain = create_stuff_documents_chain(llm,prompt)
rag_chain = create_retrieval_chain(retriever,question_answer_chain)

In [ ]:
'''from openai import OpenAI
client = OpenAI()

response = client.responses.create(
    model="gpt-5",
    input="What is acne"
)

print(response.output_text)'''

In [ ]:
'''response = rag_chain.invoke({"input":"what is the acne"})
print(response["answer"])'''